In [1]:
from tensorflow.keras import backend as K
K.clear_session()
!pip install rarfile
import os
import pandas as pd
import gdown
import rarfile
import shutil
from PIL import Image

#
#https://drive.google.com/file/d/1ATotTLVBrB3tNlj4NRcV4WzqR24UUkJ3/view?usp=drive_link
#https://drive.google.com/file/d/1OGuEqyNj4aSPSW16wavw13jmp5PXS_b3/view?usp=sharing
url = 'https://drive.google.com/uc?id=1ATotTLVBrB3tNlj4NRcV4WzqR24UUkJ3'
output_rar = '/content/SDR.rar'
gdown.download(url, output_rar, quiet=False)
# Extract the RAR file
with rarfile.RarFile(output_rar, 'r') as rar_ref:
    rar_ref.extractall('/content')
# Path to the extracted dataset folder
dataset_folder = '/content/SDR'

Downloading...
From (original): https://drive.google.com/uc?id=1ATotTLVBrB3tNlj4NRcV4WzqR24UUkJ3
From (redirected): https://drive.google.com/uc?id=1ATotTLVBrB3tNlj4NRcV4WzqR24UUkJ3&confirm=t&uuid=89737337-e7d8-49ce-a50f-e5b531f14f12
To: /content/SDR.rar
100%|██████████| 231M/231M [00:04<00:00, 48.6MB/s]


**ResNet50v2**

In [2]:
import os
import random
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define parameters
input_shape = (224, 224, 3)  # Input shape for ResNet50V2
num_classes = 2  # Adjust this to match the number of classes in your dataset
batch_size = 32
img_size = (224, 224)  # Image size
best_weights_path = '/content/Split/Save/best_model.keras'  # Path to save the best weights
train_dir = '/content/SDS/Train'  # Path to training data
test_dir = '/content/SDS/Test'  # Path to testing data

# Initialize the ResNet50V2 model with pretrained weights and custom top
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for training data
data_gen_train = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Preprocessing for test data
data_gen_test = ImageDataGenerator(rescale=1.0/255)

# Create data generators
train_data_generator = data_gen_train.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data_generator = data_gen_test.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(best_weights_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', verbose=1)

history = model.fit(
    train_data_generator,
    validation_data=test_data_generator,
    epochs=20,
    callbacks=[checkpoint, early_stopping]
)

# Unfreeze all layers and fine-tune
for layer in base_model.layers:
    layer.trainable = True

# Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
history_fine_tune = model.fit(
    train_data_generator,
    validation_data=test_data_generator,
    epochs=10,
    callbacks=[checkpoint, early_stopping]
)

# Load the best saved weights
model.load_weights(best_weights_path)

# Evaluate the model
print("Evaluating the model on test data...")
y_true = test_data_generator.classes
y_pred = np.argmax(model.predict(test_data_generator), axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1-score:", f1)

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 3340 images belonging to 2 classes.
Found 836 images belonging to 2 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.8156 - loss: 0.3997
Epoch 1: val_accuracy improved from -inf to 0.86842, saving model to /content/Split/Save/best_model.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 74s 541ms/step - accuracy: 0.8161 - loss: 0.3990 - val_accuracy: 0.8684 - val_loss: 0.3293
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 0.9055 - loss: 0.2436
Epoch 2: val_accuracy improved from 0.86842 to 0.89833, saving model to /content/Split/Save/best_model.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 66s 451ms/step - accuracy: 0.9056 - loss: 0.2434 - val_accuracy: 0.8983 - val_loss: 0.2708
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.9094 - loss: 0.2093
Epoch 3: val_accuracy did not improve from 0.89833
105/105 ━━━━━━━━━━━━━━━━━━━━ 82s 461ms/step - accuracy: 0.9094 - loss: 0.2093 - val_accuracy: 0.8732 - val_loss: 0.3190
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.9051 - loss: 0.2192
Epoch 4: val_accuracy improved fr